In [1]:
!pip install python-crfsuite

In [2]:
def generate_templates(begin=-2, end=2, min_range_length=3, max_range_length=5):
    templates = []
    for b in range(begin, end):
        for e in range(b, end+1):
            length = (e - b + 1)
            if length < min_range_length or length > max_range_length: 
                continue
            if b * e > 0:
                continue
            templates.append((b, e))
    return templates

templates = generate_templates()

print(templates)
# [(-2, 0), (-2, 1), (-2, 2), (-1, 1), (-1, 2), (0, 2)]

[(-2, 0), (-2, 1), (-2, 2), (-1, 1), (-1, 2), (0, 2)]


In [3]:
templates = generate_templates(begin=-2, end=2, min_range_length=3, max_range_length=3)

print(templates)
# [(-2, 0), (-1, 1), (0, 2)]

[(-2, 0), (-1, 1), (0, 2)]


In [4]:
class CharacterFeatureTransformer:
    def __init__(self, templates):
        self.templates = templates
    def __call__(self, chars, tags=None):
        x =[]
        for i in range(len(chars)):
            xi = []
            e_max = len(chars)
            for t in self.templates:
                b = i + t[0]
                e = i + t[1] + 1
                if b < 0 or e > e_max:
                    continue
                xi.append(('X[%d,%d]' % (t[0], t[1]), chars[b:e]))
            x.append(xi)
        return x

In [5]:
def sent_to_chartags(sent, nonspace='0', space='1'):
    chars = sent.replace(' ','')
    if not chars:
        return '', []
    tags = [nonspace]*(len(chars) - 1) + [space]
    idx = 0
    for c in sent:
        if c == ' ':
            tags[idx-1] = space
        else:
            idx += 1
    return chars, tags

sent_to_chartags('예문 입니다')
# ('예문입니다', ['0', '1', '0', '0', '1'])

('예문입니다', ['0', '1', '0', '0', '1'])

In [6]:
sent_to_chartags('')
# ('', [])

('', [])

In [7]:
def sent_to_xy(sent, feature_transformer):
    chars, tags = sent_to_chartags(sent)
    x = [['%s=%s' % (xij[0], xij[1]) for xij in xi] for xi in feature_transformer(chars, tags)]
    y = [t for t in tags]
    return x, y

x,y = sent_to_xy('예문 입니다', transformer)

NameError: name 'transformer' is not defined

In [8]:
import pycrfsuite

trainer = pycrfsuite.Trainer(verbose=False)
# possible only if len(x) == len(y)
trainer.append(x, y) 

NameError: name 'x' is not defined

In [9]:
params = {
    'max_iterations':50, # default 1000
    'c1': 0, # L1 regularization, default 1
    'c2': 1, # L2 regularization, default 2
    'feature.minfreq': 3 # minimum frequency of feature. default 0 (without pruning)
}

model_fname = 'crfsuite_spacing.model'

trainer = pycrfsuite.Trainer(verbose=False)
for doc in docs:
    x, y = sent_to_xy(doc, transformer)
    trainer.append(x, y)
trainer.set_params(params)
trainer.train(model_fname)

NameError: name 'docs' is not defined

In [10]:
!pip install pycrfsuite_spacing

In [11]:
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pprint import pprint

templates = TemplateGenerator(
    begin=-2, 
    end=2,
    min_range_length=3,
    max_range_length=3)

to_feature = CharacterFeatureTransformer(templates)

In [12]:
from pycrfsuite_spacing import sent_to_xy

x, y = sent_to_xy('이것도 너프해 보시지', to_feature)

pprint(x)
# [['X[0,2]=이것도'],
#  ['X[-1,1]=이것도', 'X[0,2]=것도너'],
#  ['X[-2,0]=이것도', 'X[-1,1]=것도너', 'X[0,2]=도너프'],
#  ['X[-2,0]=것도너', 'X[-1,1]=도너프', 'X[0,2]=너프해'],
#  ['X[-2,0]=도너프', 'X[-1,1]=너프해', 'X[0,2]=프해보'],
#  ['X[-2,0]=너프해', 'X[-1,1]=프해보', 'X[0,2]=해보시'],
#  ['X[-2,0]=프해보', 'X[-1,1]=해보시', 'X[0,2]=보시지'],
#  ['X[-2,0]=해보시', 'X[-1,1]=보시지'],
#  ['X[-2,0]=보시지']]

print(y)
# ['0', '0', '1', '0', '0', '1', '0', '0', '1']

[['X[0,2]=이것도'],
 ['X[-1,1]=이것도', 'X[0,2]=것도너'],
 ['X[-2,0]=이것도', 'X[-1,1]=것도너', 'X[0,2]=도너프'],
 ['X[-2,0]=것도너', 'X[-1,1]=도너프', 'X[0,2]=너프해'],
 ['X[-2,0]=도너프', 'X[-1,1]=너프해', 'X[0,2]=프해보'],
 ['X[-2,0]=너프해', 'X[-1,1]=프해보', 'X[0,2]=해보시'],
 ['X[-2,0]=프해보', 'X[-1,1]=해보시', 'X[0,2]=보시지'],
 ['X[-2,0]=해보시', 'X[-1,1]=보시지'],
 ['X[-2,0]=보시지']]
['0', '0', '1', '0', '0', '1', '0', '0', '1']


In [14]:
from pycrfsuite_spacing import PyCRFSuiteSpacing

correct = PyCRFSuiteSpacing(
    to_feature,
    feature_minfreq=5, 
    max_iterations=100,
    l1_cost=1.0,
    l2_cost=1.0)

model_path = 'package_test.crfsuite'
correct.train("https://lovit.github.io/assets/resources/134963_norm.txt", model_path)

In [15]:
correct('이건진짜좋은영화라라랜드진짜좋은영화')

'이 건 진 짜 좋 은 영 화 라 라 랜 드 진 짜 좋 은 영 화'

In [16]:
correct.correct('이건진짜좋은영화라라랜드진짜좋은영화')

'이 건 진 짜 좋 은 영 화 라 라 랜 드 진 짜 좋 은 영 화'

In [17]:
model = PyCRFSuiteSpacing(to_feature)
model.load_tagger(model_path)

In [18]:
model('이건진짜좋은영화라라랜드진짜좋은영화')

'이 건 진 짜 좋 은 영 화 라 라 랜 드 진 짜 좋 은 영 화'